In [18]:
def salvar_html(html, filename):
    """Salva o resultado de uma requisição para visualização (sem css) no browser
    """
    with open(filename,'w') as myf:
        myf.write(html)
    

### Setup do browser

In [1]:
import mechanize
from urllib import urlencode
from bs4 import BeautifulSoup as bsoup
import sqlite3
#Browser
driver = mechanize.Browser()
driver.set_handle_robots(False)
driver.addheaders = [('User-agent',
                    'Mozilla/5.0 (compatible; ABrowse 0.4; Syllable)')]

clear_cookies = driver._ua_handlers['_cookies'].cookiejar.clear
#URLs
base_url = "https://www.ncbi.nlm.nih.gov"

### Encontrando os artigos na primeira página

In [8]:
### Pesquisar
pesquisa = {'term':'risk AND machine learning'}
clear_cookies()
resposta = driver.open(base_url + '/pmc/', urlencode(pesquisa))

### Pegar resultados
soup = bsoup(resposta.read(), "html5lib")
#Título da página
print soup.findAll('title')[0].text
# Número da página
print soup.findAll('input',id='pageno')[0]['value'] + '\n'
#Títulos dos papers encontrados
links = soup.findAll("div", {"class":"title"})
print '\n\n'.join([link.text for link in links[:5]])
articles = links

risk AND machine learning - PMC - NCBI
1

Progressive sampling-based Bayesian optimization for efficient and automatic machine learning model selection

MLBCD: a machine learning tool for big clinical data

Can machine-learning improve cardiovascular risk prediction using routine clinical data?

Machine Learning and Radiology

Accuracy of automated classification of major depressive disorder as a function of symptom severity


### Encontrando os artigos depois da primeira página

In [10]:
from imports_pmc import _querystring, _pageno
### Pesquisar, setando o request com o número da página
_querystring[_pageno] = 3
resposta = driver.open("https://www.ncbi.nlm.nih.gov/pmc/",
                       urlencode(_querystring)
                      )

### Pegar resultados
soup = bsoup(resposta.read(), "html5lib")
#Título da página
print soup.findAll('title')[0].text
# Número da página
print soup.findAll('input',id='pageno')[0]['value'] + '\n'
#Títulos dos papers encontrados
links = soup.findAll("div", {"class":"title"})
print '\n\n'.join([link.text[:65] + '...' for link in links[:3]])

risk AND machine learning - PMC - NCBI
3

Machine Learning Algorithms Outperform Conventional Regression Mo...

Using Machine Learning and Natural Language Processing Algorithms...

A Cognitive Machine Learning Algorithm for Cardiac Imaging: A Pil...


In [28]:
[link.findAll('a', href=True)[0]['href'] for link in links[:3]]

[u'/pmc/articles/PMC3335189/',
 u'/pmc/articles/PMC4516296/',
 u'/pmc/articles/PMC4336000/']

### Pegando as keywords

e mais algum parágrafo

In [46]:
for article in articles[:1]:
    url = base_url + article.findAll('a', href=True)[0]['href']
    resposta = driver.open(url)
    soup = bsoup(resposta.read(), "html5lib")
    #Título da página
    print soup.findAll('title')[0].text[:65] + '...\n'
    try:
        print soup.findAll('span',{"class":"kwd-text"})[0].text+ '...\n'
    except:
        pass
    print soup.findAll('p',{"class":"p p-first-last"})[0].text+ '...\n'
    
    print '\n\n'
    

Progressive sampling-based Bayesian optimization for efficient an...

Automatic machine learning model selection, Bayesian optimization, Progressive sampling, Clinical big data...

Machine learning is broadly used for clinical data analysis. Before training a model, a machine learning algorithm must be selected. Also, the values of one or more model parameters termed hyper-parameters must be set. Selecting algorithms and hyper-parameter values requires advanced machine learning knowledge and many labor-intensive manual iterations. To lower the bar to machine learning, miscellaneous automatic selection methods for algorithms and/or hyper-parameter values have been proposed. Existing automatic selection methods are inefficient on large data sets. This poses a challenge for using machine learning in the clinical big data era....






### Cruzando as palavras-chave

ou seja, fazendo a busca de par em par de palavras-chave e salvando os resultados

In [38]:
from imports_pmc import _querystring, _pageno
from datetime import datetime

domain_specific = [
    'Coronary Artery Disease',
    'Arterial Coronary Syndrome',
    'Patient Assessment',
    'Risk Stratification',
    'Risk Score',
    'Angiography',
    'Tomography'
]
keywords = [
    'Image processing',
    'image segmentation',
    'Artificial Intelligence',
    'Deep Learning'
]
articles_per_page = 5
page_depth = 2

stems = {}

driver = mechanize.Browser()
driver.set_handle_robots(False)
driver.addheaders = [('User-agent',
                    'Mozilla/5.0 (compatible; ABrowse 0.4; Syllable)')]

clear_cookies = driver._ua_handlers['_cookies'].cookiejar.clear

for domain in domain_specific[:1]:
    for keyword in keywords[:1]:
        term = ' AND '.join((domain, keyword))
        research = {'term': term}
        stem = []
        append = stem.append
        ###
        ### Primeira página
        ###
        clear_cookies()
        print '[*]resgatando titulos de',term,
        resposta = driver.open(base_url + '/pmc/', urlencode(research))
        print 'ok'
        # Pegar resultados
        soup = bsoup(resposta.read(), "html5lib")
        # Articles encontrados
        articles = soup.findAll("div", {"class":"title"})
        
        for article in articles[:articles_per_page]:
            # Reunir conteúdo de cada paper
            content = {}
            url = base_url + article.findAll('a', href=True)[0]['href']
            print '[-]lendo ',article.text[:65].replace('\n',' ') +'(...)',
            resposta = driver.open(url)
            print 'ok'
            soup = bsoup(resposta.read(), "html5lib")
            content['title'] = soup.findAll('title')[0].text
            content['url'] = url
            
            try:
                content['kwd'] = soup.findAll('span',{"class":"kwd-text"})[0].text
            except:
                content['kwd'] = '-'
            try:
                content['p'] = soup.findAll('p',{"class":"p p-first-last"})[0].text
                content['p'] += soup.findAll('p',{"class":"p p-first-last"})[1].text
            except:
                content['p'] = '-'
            
            append(content)
            
        ###
        ### Segunda página
        ###
        for page in range(2,page_depth + 1):
            _querystring[_pageno] = page
            resposta = driver.open(base_url + '/pmc/', urlencode(_querystring))
            
            # Pegar resultados
            soup = bsoup(resposta.read(), "html5lib")
            # Articles encontrados
            articles = soup.findAll("div", {"class":"title"})
            
            for article in articles[:articles_per_page]:
                # Reunir conteúdo de cada paper
                content = {}
                url = base_url + article.findAll('a', href=True)[0]['href']
                print '[-]lendo ',article.text[:65].replace('\n',' ') +'(...)',
                resposta = driver.open(url)
                print 'ok'
                soup = bsoup(resposta.read(), "html5lib")
                content['title'] = soup.findAll('title')[0].text
                content['url'] = url
                
                try:
                    content['kwd'] = soup.findAll('span',{"class":"kwd-text"})[0].text
                except:
                    content['kwd'] = '-'
                try:
                    content['p'] = soup.findAll('p',{"class":"p p-first-last"})[0].text
                    content['p'] += soup.findAll('p',{"class":"p p-first-last"})[1].text
                except:
                    content['p'] = '-'

                append(content)
                
        stems[term] = stem
    filename = "stems" + str(datetime.now()).replace(' ','-') + '.txt'
    with open(filename, 'w') as myf:
        myf.write(str(stems))
    print '[*] fim: resultados salvos em',filename
            
            
            

[*]resgatando titulos de Coronary Artery Disease AND Image processing ok
[-]lendo  Cardiac magnetic resonance and computed tomography angiography fo(...) ok
[-]lendo  Noninvasive FFR derived from coronary CT angiography in the manag(...) ok
[-]lendo  Assessment of stable coronary artery disease by cardiovascular ma(...) ok
[-]lendo  Total Coronary Atherosclerotic Plaque Burden Assessment by CT Ang(...) ok
[-]lendo  Visualization of Coronary Wall Atherosclerosis in Asymptomatic Su(...) ok
[-]lendo  Prevalence of Coronary Artery Disease Evaluated by Coronary CT An(...) ok
[-]lendo  The coronary circulation in exercise training(...) ok
[-]lendo  CT angiography in the diagnosis of cardiovascular disease: a tran(...) ok
[-]lendo  Coronary CT Angiography in Heavily Calcified Coronary Arteries: I(...) ok
[-]lendo  Coronary CT Angiography in the Quantitative Assessment of Coronar(...) ok
[*] fim: resultados salvos em stems2017-11-29-22:14:23.934958.txt


### Selecionando artigos interativamente

In [ ]:
import webbrowser
from ast import literal_eval
import re

with open(filename,'r') as myf:
    print '[*] Extraindo artigos de', filename
    s = myf.read()
    stems = literal_eval(s)
for key in stems.keys():
    i = 0
    while i < len(stems[key]):
        article = stems[key][i]
        print article['title'],'\n'
        print article['kwd'],'\n'
        print article['p'],'\n'
        
        apagar = re.compile("(apa|del|rem)",re.IGNORECASE)
        url = re.compile("(url|link|ender|site|abr)",re.IGNORECASE)
        proximo = re.compile("(con|ava|pro|nex)",re.IGNORECASE)
        sair = re.compile("(sai|lea|exi|fin)",re.IGNORECASE)
        
        match = False
        while not match:
            act = raw_input("Ação ")
            if apagar.match(act):
                print 'apagando'
                stems[key].remove(article)
                match = True
                i -= 1
            elif url.match(act):
                print 'abrindo'
                i -= 1
                webbrowser.open(article['url'])
                match = True
            elif proximo.match(act) or act == '':
                print 'continuando'
                match = True
                continue
            elif sair.match(act):
                i = len(stems[key])
                match = True
            else:
                print 'invalido!'
        i += 1
    filename = "stems" + str(datetime.now()).replace(' ','-') + '.txt'
    with open(filename, 'w') as myf:
        myf.write(str(stems))
        print '[*] resultados salvos em',filename

[*] Extraindo artigos de stems2017-11-29-22:14:23.934958.txt
Cardiac magnetic resonance and computed tomography angiography for clinical imaging of stable coronary artery disease. Diagnostic classification and risk stratification 

coronary artery disease, atherosclerotic plaque, coronary computed tomography, cardiac magnetic resonance, risk stratification 

Despite advances in the pharmacologic and interventional treatment of coronary artery disease (CAD), atherosclerosis remains the leading cause of death in Western societies. X-ray coronary angiography has been the modality of choice for diagnosing the presence and extent of CAD. However, this technique is invasive and provides limited information on the composition of atherosclerotic plaque. Coronary computed tomography angiography (CCTA) and cardiac magnetic resonance (CMR) have emerged as promising non-invasive techniques for the clinical imaging of CAD. Hereby, CCTA allows for visualization of coronary calcification, lumen narro

In [32]:
filename

'stems2017-11-29-21:55:15.401655.txt'